<a href="https://colab.research.google.com/github/manojkhara/data-science-lab/blob/main/week-3/pytorch/4%20RNN%20using%20pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.0.1 torchtext==0.15.2 portalocker==2.8.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
#1 get data
from torchtext.datasets import IMDB
train_iter = IMDB(split= "train")
test_iter = IMDB(split= "test")


In [ ]:
import numpy as np

targets = []
for x in train_iter:
    targets.append(x[0])

print(len(targets))
print(np.unique(targets, return_counts=True))

25000
(array([1, 2]), array([12500, 12500]))


In [ ]:
targets = []
for x in test_iter:
    targets.append(x[0])

print(len(targets))
print(np.unique(targets, return_counts=True))

25000
(array([1, 2]), array([12500, 12500]))


In [ ]:
train_loader = DataLoader(train_iter, batch_size=16, shuffle=True)

In [ ]:
train_loader

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
for labels, features in train_iter:
  print(labels)
  print(features)
  break

1
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [ ]:
labels

1

In [ ]:
features

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
#2 Tokenizer Function
from torchtext.data.utils import  get_tokenizer
tokenizer = get_tokenizer("basic_english")

In [ ]:
#3 Build a Vocabulary
from torchtext.vocab import build_vocab_from_iterator
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

/usr/local/lib/python3.11/dist-packages/torch/utils/data/datapipes/iter/combining.py:297: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [ ]:
len(vocab)

100684

In [ ]:
vocab["<pad>"]

0

In [ ]:
vocab["i"]

12

In [ ]:
vocab["controversy"]

7332

In [ ]:
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)


In [ ]:
# next(yielding)
# next(yielding)
# next(yielding)

In [ ]:
vocab(tokenizer("this movie was great"))

[14, 18, 17, 148]

In [ ]:
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence

In [ ]:
label_changer = lambda label : 0 if  label==1 else 1

In [ ]:
#4 Function to convert a batch of text data into numbers using the tokenizer and vocabulary
# [this movie was great] ---> [0, 345, 56, 1993]

def collate_batch(batch):
  label_list, text_list = [], []
  for label, text in batch:
    label_list.append(label_changer(label))
    preprocessed_text = torch.tensor(vocab(tokenizer(text)))
    text_list.append(preprocessed_text)

  label_list = torch.tensor(label_list)

  text_list= pad_sequence(text_list, batch_first=True)
  return label_list, text_list


In [ ]:
#5 DataLoader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_iter, batch_size=16, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_iter, batch_size=16, shuffle=True, collate_fn=collate_batch)


In [ ]:
tensor = torch.randn(4, 16, 5) # 4 movies , 16 words , 5 size embeding
tensor


tensor([[[ 1.7452e+00,  1.1915e+00,  1.0513e-01,  1.5350e+00,  6.8558e-01],
         [ 5.4954e-01,  2.3717e-02, -5.2975e-03, -1.5760e+00, -1.9034e-01],
         [-8.6543e-02, -6.3622e-02,  5.0911e-01, -7.9676e-01,  1.0452e+00],
         [ 7.1784e-01, -7.7452e-01,  5.6181e-01, -6.6701e-01,  3.7154e-01],
         [ 1.1136e+00, -1.3947e+00,  1.1086e+00, -3.6365e-01,  1.0847e+00],
         [-2.0389e-01,  4.7250e-01,  2.5203e+00,  8.6000e-01, -1.0914e+00],
         [ 5.2231e-01, -2.9732e-01,  1.3578e+00,  1.6107e+00,  5.7232e-01],
         [ 4.3576e-01,  2.0664e-01,  2.7528e-01,  2.3408e+00,  1.1571e-01],
         [-7.3007e-01, -2.1055e-01,  3.1924e-01, -1.6216e+00,  1.1819e+00],
         [-8.5004e-01, -7.4053e-01,  1.3104e-01, -4.5072e-01, -7.9320e-01],
         [ 5.2948e-02, -1.1229e-01, -8.9678e-02, -7.2127e-01,  3.6742e-01],
         [ 1.2762e+00, -1.4445e+00, -9.0989e-02,  1.6072e-01,  1.8419e-01],
         [ 3.7475e-02, -2.3541e-01,  1.0243e+00, -3.2463e-01,  1.1583e+00],
         [-1

In [ ]:
rnn_layer =   nn.RNN(input_size = 5, hidden_size = 3, num_layers = 2, batch_first = True)
states, output = rnn_layer(tensor)

In [ ]:
states

tensor([[[-0.2309, -0.4207, -0.5205],
         [-0.3100, -0.6584, -0.1300],
         [-0.3125, -0.4885, -0.1085],
         [-0.1973, -0.3946, -0.3020],
         [-0.1725, -0.6281, -0.2283],
         [-0.4683, -0.3084, -0.3193],
         [-0.3994, -0.3638, -0.1657],
         [-0.4580, -0.4381, -0.0948],
         [-0.4366, -0.6288,  0.3149],
         [-0.4528, -0.0827, -0.1171],
         [-0.3233, -0.3484, -0.1343],
         [-0.2243, -0.3750, -0.3576],
         [-0.3701, -0.7493,  0.1338],
         [-0.4788, -0.6296,  0.3344],
         [-0.2813, -0.3121,  0.0136],
         [-0.3861, -0.3789, -0.1816]],

        [[-0.1640, -0.6445, -0.2545],
         [-0.4716, -0.2812, -0.4293],
         [-0.4749, -0.5219, -0.0224],
         [-0.3491, -0.6711,  0.3021],
         [-0.3708, -0.5570,  0.1871],
         [-0.3204,  0.0849, -0.4464],
         [-0.1745, -0.3761, -0.4716],
         [-0.3777, -0.3244, -0.5115],
         [-0.3564, -0.7523,  0.1945],
         [-0.1712, -0.5166, -0.0573],
         [

In [ ]:
output[-1]

tensor([[-0.3861, -0.3789, -0.1816],
        [-0.4020, -0.1781, -0.3602],
        [-0.4204, -0.5418, -0.1828],
        [-0.4921, -0.7082,  0.4825]], grad_fn=<SelectBackward0>)

In [ ]:
#6 Define RNN-based model


class senitmentModel(nn.Module):
  def __init__(self, ):
    super().__init__()
    self.embedding = nn.Embedding(num_embeddings=len(vocab), embedding_dim=64)
    self.rnn = nn.RNN(input_size=64, hidden_size=32, num_layers=1, batch_first=True)
    self.linear = nn.Linear(in_features=32, out_features=2) # binary classification

  def forward(self,text_data):
    embedings = self.embedding(text_data)
    states, outputs = self.rnn(embedings)
    x =  self.linear(outputs[-1])
    return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#7 Setup training
torch.manual_seed(42)

# define the model
model = senitmentModel()
model.to(device)

optimizer = optim.SGD(model.parameters(), lr = 0.01)

EPOCHS = 32


In [ ]:
#8 Train loop

model.train()

for epoch in range(EPOCHS):

  loss_per_epoch = 0

  for labels, features in train_loader:
    features = features.to(device)
    labels = labels.to(device)


    # forward pass
    y_pred = model.forward(features)

    # loss computation
    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(y_pred, labels)


    #make gradients zero
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    #weight updates
    optimizer.step()

    loss_per_epoch += loss

  print("loss per epoch =", loss_per_epoch)

loss per epoch = tensor(855.6309, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(862.5518, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(861.1894, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(869.3461, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(872.0623, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(869.7094, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(863.3782, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(864.2837, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(866.9114, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(868.6473, device='cuda:0', grad_fn=<AddBackward0>)
loss per epoch = tensor(867.5347, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [ ]:
# evaluation on train data
total = 0
correct = 0

with torch.no_grad():

  for batch_labels, batch_features in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

AttributeError: 'tuple' object has no attribute 'to'

In [ ]:
test_iter = IMDB(split= "test")
test_loader = DataLoader(test_iter, batch_size=16, shuffle=True, collate_fn=collate_batch)

In [ ]:
#9 evaluate on test data
model.eval()

with torch.no_grad():
  loss_per_epoch = 0
  for labels, features in test_loader:
    features = features.to(device)
    labels = labels.to(device)


    # forward pass
    y_pred = model.forward(features)

    # loss computation
    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(y_pred, labels)

    loss_per_epoch += loss

  print("loss per epoch =", loss_per_epoch)

loss per epoch = tensor(4680.2925)


In [ ]:
# Accuracy on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_labels, batch_features in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.49956
